## XRLint CLI

---

In [1]:
!xrlint --help

Usage: xrlint [OPTIONS] [FILES]...

  Validate the given dataset FILES.

  Reads configuration from './xrlint_config.*' if such file exists and unless
  '--no-config-lookup' is set or '--config' is provided. It then validates
  each dataset in FILES against the configuration. The default dataset patters
  are '**/*.zarr' and '**/.nc'. FILES may comprise also directories or URLs.
  The supported URL protocols are the ones supported by xarray. Using remote
  protocols may require installing additional packages such as S3Fs
  (https://s3fs.readthedocs.io/) for the 's3' protocol.

  If a directory is provided that not matched by any file pattern, it will be
  traversed recursively. The validation result is dumped to standard output if
  not otherwise stated by '--output-file'. The output format is 'simple' by
  default. Other inbuilt formats are 'json' and 'html' which you can specify
  using the '--format' option.

Options:
  --no-config-lookup      Disable use of default configuration fr

In [2]:
import os

os.getcwd()

'C:\\Users\\Norman\\Projects\\xrlint\\notebooks'

In [3]:
!xrlint --init

Error: file xrlint_config.yaml already exists.


In [4]:
from mkdataset import make_dataset, make_dataset_with_issues

make_dataset().to_zarr("valid.zarr", mode="w", encoding=dict(
    sst=dict(write_empty_chunks=False), 
    sst_anomaly=dict(write_empty_chunks=False),
))
make_dataset_with_issues().to_zarr("invalid.zarr", mode="w")

In [5]:
!xrlint --no-color valid.zarr invalid.zarr 


valid.zarr - ok

invalid.zarr:
dataset                           warn   Missing attribute 'title'.                                  content-desc
dataset                           warn   Missing attribute 'history'.                                content-desc
dataset.data_vars['sst']          warn   Missing attribute 'institution'.                            content-desc
dataset.data_vars['sst']          warn   Missing attribute 'source'.                                 content-desc
dataset.data_vars['sst']          warn   Missing attribute 'references'.                             content-desc
dataset.data_vars['sst']          warn   Missing attribute 'comment'.                                content-desc
dataset.data_vars['sst_anomaly']  warn   Missing attribute 'institution'.                            content-desc
dataset.data_vars['sst_anomaly']  warn   Missing attribute 'source'.                                 content-desc
dataset.data_vars['sst_anomaly']  warn   Missing attribu

In [6]:
!xrlint --no-color --rule "content-desc: off" --rule "no-empty-chunks: off" valid.zarr invalid.zarr 


valid.zarr - ok

invalid.zarr:
dataset                       warn   Missing attribute 'Conventions'.                            conventions
dataset.coords['y']           error  Attribute 'standard_name' should be 'latitude', was None.   lat-coordinate
dataset.coords['y']           error  Attribute 'axis' should be 'Y', was 'y'.                    lat-coordinate
dataset.coords['x']           error  Attribute 'units' should be 'degrees_east', was 'degrees'.  lon-coordinate
dataset.coords['x']           error  Attribute 'axis' should be 'X', was 'x'.                    lon-coordinate
dataset.attrs                 warn   Missing metadata, attributes are empty.                     no-empty-attrs
dataset.coords['time']        error  Invalid encoding 'units': 'days since 2020-01-01 UTC'.      time-coordinate
dataset.data_vars['sst_avg']  warn   Missing attribute 'standard_name'.                          var-desc
dataset.data_vars['sst_avg']  warn   Missing attribute 'long_name'.             

In [7]:
!xrlint --no-color --rule "content-desc: off" --rule "var-desc: [error, {'attrs': ['description']}]" valid.zarr invalid.zarr 


valid.zarr:
dataset.data_vars['sst']          error  Missing attribute 'description'.  var-desc
dataset.data_vars['sst_anomaly']  error  Missing attribute 'description'.  var-desc

invalid.zarr:
dataset                           warn   Missing attribute 'Conventions'.                            conventions
dataset.coords['y']               error  Attribute 'standard_name' should be 'latitude', was None.   lat-coordinate
dataset.coords['y']               error  Attribute 'axis' should be 'Y', was 'y'.                    lat-coordinate
dataset.coords['x']               error  Attribute 'units' should be 'degrees_east', was 'degrees'.  lon-coordinate
dataset.coords['x']               error  Attribute 'axis' should be 'X', was 'x'.                    lon-coordinate
dataset.attrs                     warn   Missing metadata, attributes are empty.                     no-empty-attrs
dataset.coords['time']            error  Invalid encoding 'units': 'days since 2020-01-01 UTC'.      time-coord

In [8]:
!xrlint valid.zarr invalid.zarr -f json

{
  "results": [
    {
      "config_object": {
        "plugins": {
          "__core__": "xrlint.plugins.core:export_plugin"
        },
        "rules": {
          "content-desc": 1,
          "conventions": 1,
          "coords-for-dims": 2,
          "grid-mappings": 2,
          "lat-coordinate": 2,
          "lon-coordinate": 2,
          "no-empty-attrs": 1,
          "no-empty-chunks": 0,
          "time-coordinate": 2,
          "var-desc": 1,
          "var-flags": 2,
          "var-units": 1
        }
      },
      "file_path": "valid.zarr",
      "messages": []
    },
    {
      "config_object": {
        "plugins": {
          "__core__": "xrlint.plugins.core:export_plugin"
        },
        "rules": {
          "content-desc": 1,
          "conventions": 1,
          "coords-for-dims": 2,
          "grid-mappings": 2,
          "lat-coordinate": 2,
          "lon-coordinate": 2,
          "no-empty-attrs": 1,
          "no-empty-chunks": 0,
          "time-coordinate"

In [9]:
!xrlint --print-config valid.zarr

{
  "plugins": {
    "__core__": "xrlint.plugins.core:export_plugin"
  },
  "rules": {
    "content-desc": 1,
    "conventions": 1,
    "coords-for-dims": 2,
    "grid-mappings": 2,
    "lat-coordinate": 2,
    "lon-coordinate": 2,
    "no-empty-attrs": 1,
    "no-empty-chunks": 0,
    "time-coordinate": 2,
    "var-desc": 1,
    "var-flags": 2,
    "var-units": 1
  }
}
